# Importing Libraries and Helper Classes and Functions

In [2]:
import sys
import os
import json
import numpy as np
from tqdm import tqdm
import torch
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
from utils_data import  generateTargetDataBuySide, generateTargetDataSellSide, getTechnicalIndicators, normalize_dataframe_with_mean_std
from utils_data import UpstoxStockDataFetcher

# Loading Data

In [3]:
fetcher = UpstoxStockDataFetcher()
stock_symbol = "SUZLON"
start_date = "2025-06-11"
end_date = "2025-06-17"
df = fetcher.get_stock_data(stock_symbol, start_date, end_date)
df = getTechnicalIndicators(df)
target_buy = generateTargetDataBuySide(df,1.005,0.99)
target_sell = generateTargetDataSellSide(df,0.995,1.01)



Processing Sell Side Data: 100%|██████████| 1473/1473 [00:17<00:00, 82.02it/s] 


# Data Analysis

In [4]:
count1 = len(target_buy[target_buy['action'].isin(['End of Day'])])
count2 = len(target_buy[target_buy['action'].isin(['Target Hit'])])
count3 = len(target_buy[target_buy['action'].isin(['Stop Loss Hit'])])

print(f"End of Day: {count1}")
print(f"Target Hit: {count2}")
print(f"Stop Loss Hit: {count3}")

End of Day: 384
Target Hit: 634
Stop Loss Hit: 455


In [5]:
count4 = len(target_sell[target_sell['action'].isin(['End of Day'])])
count5 = len(target_sell[target_sell['action'].isin(['Target Hit'])])
count6 = len(target_sell[target_sell['action'].isin(['Stop Loss Hit'])])

print(f"End of Day: {count4}")
print(f"Target Hit: {count5}")
print(f"Stop Loss Hit: {count6}")

End of Day: 327
Target Hit: 870
Stop Loss Hit: 276


# Normalizing and Storing Params

In [6]:
df_normalized, norm_param = normalize_dataframe_with_mean_std(df)

# Storing Norm parameters to be used later for inference
with open("C:/Users/srija/Assignment/Trading/json_files/suzlon_14_june_norm_params.json", "w") as f:
    json.dump(norm_param, f)

df_normalized

c:\Users\srija\Assignment\Trading\utils_data.py:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normalized_df[col].iloc[i] = normalized_df[col].iloc[i].hour*60 + normalized_df[col].iloc[i].minute-540
c:\Users\srija\Assignment\Trading\utils_data.py:207: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '42' has dtype incompatible with datetime64[ns, UTC+05:30], please explicitly cast to a compatible dtype first.
  normalized_df[col].iloc[i] = normalized_df[col].iloc[i].hour*60 + normalized_df[col].iloc[i].minute-540


,time,open,high,low,close,volume,MA50,RSI,MACD,BB_upper,BB_lower,ADX,CCI,ATR,ROC,OBV
0,42,68.35,68.40,68.33,68.36,0.121618,1.699062,0.803805,0.582877,1.706032,1.659184,-0.226936,0.475535,1.397213,-0.016104,2.059316
1,43,68.37,68.38,68.35,68.36,-0.305492,1.707238,0.803805,0.565125,1.700603,1.677476,-0.280383,0.432600,1.203042,0.314196,2.086973
2,44,68.38,68.38,68.31,68.33,-0.213411,1.705450,0.652320,0.518791,1.676647,1.715444,-0.387618,0.023131,1.084364,0.093899,2.053423
3,45,68.33,68.33,68.26,68.30,-0.217055,1.699062,0.502135,0.450592,1.674890,1.719186,-0.547933,-0.886931,0.974164,0.093899,2.020106
4,46,68.30,68.31,68.26,68.29,-0.350719,1.696251,0.451772,0.384578,1.658987,1.721851,-0.696796,-0.936793,0.841023,0.204112,1.995343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,385,65.17,65.19,65.13,65.15,0.027499,-0.360832,-1.053319,-0.781498,-0.284325,-0.491373,0.344517,-0.842435,-0.064929,-1.626327,0.863067
1469,386,65.15,65.19,65.12,65.19,0.401421,-0.370541,-0.782112,-0.798893,-0.299030,-0.505746,0.416083,-0.736377,-0.093037,-0.749543,0.935962
1470,387,65.19,65.19,65.13,65.15,0.848630,-0.385360,-0.971305,-0.838671,-0.311632,-0.522809,0.482538,-0.716707,-0.134543,-0.482144,0.834449
1471,388,65.15,65.20,65.13,65.14,-0.005158,-0.395836,-1.018158,-0.867726,-0.327410,-0.537547,0.521677,-0.677065,-0.157678,-0.711944,0.787572


# Loading Models

In [7]:
from Models.DQN import DQN,DQNAgent
from trading_environment import StockTradingEnv

policy_net = DQN(16, 3)
target_net = DQN(16, 3)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

DQN(
  (fc1): Linear(in_features=16, out_features=1440, bias=True)
  (fc2): Linear(in_features=1440, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=128, bias=True)
  (fc6): Linear(in_features=128, out_features=64, bias=True)
  (fc7): Linear(in_features=64, out_features=32, bias=True)
  (fc8): Linear(in_features=32, out_features=3, bias=True)
)

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

policy_net.to(device)
target_net.to(device)


DQN(
  (fc1): Linear(in_features=16, out_features=1440, bias=True)
  (fc2): Linear(in_features=1440, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=128, bias=True)
  (fc6): Linear(in_features=128, out_features=64, bias=True)
  (fc7): Linear(in_features=64, out_features=32, bias=True)
  (fc8): Linear(in_features=32, out_features=3, bias=True)
)

In [9]:
def get_state(df, current_step):
    row = df.iloc[current_step]
    state = np.array([
        row['time'],
        row['open'],
        row['high'],
        row['low'],
        row['close'],
        row['volume'],
        row['MA50'],
        row['RSI'],
        row['MACD'],
        row['BB_upper'],
        row['BB_lower'],
        row['ADX'],
        row['CCI'],
        row['ATR'],
        row['ROC'],
        row['OBV']
    ], dtype=np.float32)
    return state


In [10]:
save_folder = r"C:\Users\srija\Assignment\Trading\Models\trained_models\suzlon_14_june"
model_path = r"C:\Users\srija\Assignment\Trading\Models\trained_models\suzlon_14_june\suzlon_14_june_6981.pth"
policy_net.load_state_dict(torch.load(model_path))
policy_net.train()
env = StockTradingEnv(df_normalized)
agent = DQNAgent(env, policy_net, target_net)

# Defining Reward Function

In [11]:
def calculate_optimized_scalping_reward(delay, action_type, success_base_reward=1500, 
                                     failure_base_penalty=1000, min_delay_threshold=60, 
                                     max_reward=2500, decay_rate=0.3,
                                     opportunity_cost_factor=0.2,
                                     missed_opp_multiplier=2.0,
                                     consecutive_successes=0, 
                                     consecutive_success_bonus=0.15):
    """
    Comprehensive reward function optimized for scalping.
    """
    delay = delay/60
    if action_type == 'success':
        # Delay-dependent base reward scaling
        if delay <= min_delay_threshold:
            base_reward = max_reward - (max_reward - success_base_reward) * (delay / min_delay_threshold)
        else:
            base_reward = success_base_reward
        
        # Apply exponential decay
        reward = base_reward * np.exp(-decay_rate * delay)
        
        # Apply opportunity cost
        opportunity_cost = opportunity_cost_factor * delay * success_base_reward
        opportunity_cost = min(opportunity_cost, reward * 0.8)
        reward = reward - opportunity_cost
        
        # Apply sequential bonus
        if consecutive_successes > 0:
            sequential_bonus = reward * (consecutive_success_bonus * consecutive_successes)
            reward += sequential_bonus
        
        return reward
    
    elif action_type == 'failure':
        # Standard penalty with exponential decay
        penalty = -failure_base_penalty * np.exp(-decay_rate * delay)
        
        # Add opportunity cost to penalty
        opportunity_cost = opportunity_cost_factor * delay * failure_base_penalty
        penalty = penalty - opportunity_cost
        
        return penalty
    
    elif action_type == 'missed_opportunity':
        # Enhanced penalty for missed opportunities
        missed_penalty = -failure_base_penalty * missed_opp_multiplier * np.exp(-decay_rate * delay)
        return missed_penalty
    
    elif action_type == 'no_action':
        # Reward for correctly staying out of the market
        return 100


# Training the Model 

In [ ]:
num_episodes = 9000

for episode in range(6982,num_episodes):
    # state = env.reset()
    total_reward = 0
    number_trans = 0
    wins =0
    lose = 0
    defeat =0
    consecutive_success = 0
    pbar = tqdm(total = len(df_normalized))
    step =0
    next_step = 0

    while step<len(df_normalized):
    
        
        state = get_state(df_normalized,step)
        action = agent.select_action(state)
        done = False
        # print(action)
       
        if action ==1: ## BUY
            next_state = target_buy.iloc[step]
            
            next_state_index = next_state["next_state_index"]
            
            next_state2 = df_normalized.iloc[next_state_index].copy()

            reward = 0
            if(next_state['action']=="Target Hit"):
                wins +=1
                consecutive_success+=1
                # reward = 1000/(target_buy.iloc[step]['delay']+1)
                reward = calculate_optimized_scalping_reward(delay=target_buy.iloc[step]['delay'],action_type="success",consecutive_successes=consecutive_success)
                
              
            elif next_state['action']=="Stop Loss Hit":
                defeat +=1
                consecutive_success=0
                # reward = -1000/(target_buy.iloc[step]['delay']+1)
                reward = calculate_optimized_scalping_reward(delay=target_buy.iloc[step]['delay'],action_type="failure",consecutive_successes=consecutive_success)
                
                
            elif next_state['action']=="End of Day":
                lose+=1
                consecutive_success=0
                done = True
                # reward = -50
                reward = calculate_optimized_scalping_reward(delay=target_buy.iloc[step]['delay'],action_type="failure",consecutive_successes=consecutive_success)
                
               
            reward = float(reward)  # Convert to scalar float
            
            next_state2 = np.array(next_state2.values, dtype=np.float32)
            agent.store_transition(state, action, reward, next_state2, done)
            agent.update_policy()
            number_trans +=1
            next_step = next_state_index+1
        
        if action==2: ## Sell  short
            next_state = target_sell.iloc[step]
            next_state_index = next_state["next_state_index"]
            
            next_state2 = df_normalized.iloc[next_state_index].copy()
            reward = 0
            if(next_state['action']=="Target Hit"):
                wins +=1
                consecutive_success+=1
                # reward = 1000/(target_sell.iloc[step]['delay']+1)
                reward = calculate_optimized_scalping_reward(delay=target_sell.iloc[step]['delay'],action_type="success",consecutive_successes=consecutive_success)
                
               
            elif next_state['action']=="Stop Loss Hit":
                consecutive_success=0
                defeat +=1
                # reward = -1000/(target_sell.iloc[step]['delay']+1)
                reward = calculate_optimized_scalping_reward(delay=target_sell.iloc[step]['delay'],action_type="failure",consecutive_successes=consecutive_success)
                
               
            elif next_state['action']=="End of Day":
                consecutive_success=0
                lose+=1
                done =True
                # reward = -50
                reward = calculate_optimized_scalping_reward(delay=target_sell.iloc[step]['delay'],action_type="failure",consecutive_successes=consecutive_success)
                
                
            reward = float(reward)  # Convert to scalar float
           
            next_state2 = np.array(next_state2.values, dtype=np.float32)
            
            agent.store_transition(state, action, reward, next_state2, done)
            agent.update_policy()
            number_trans +=1
            next_step = next_state_index+1
        if action ==0:
            buy_side = target_buy.iloc[step].copy()
            sell_side = target_sell.iloc[step].copy()
            if buy_side['action']=="Target Hit":
                # reward = -1000/(target_buy.iloc[step]['delay']+1)
                reward = calculate_optimized_scalping_reward(delay=target_buy.iloc[step]['delay'],action_type="missed_opportunity",consecutive_successes=consecutive_success)     
            elif sell_side['action']=="Target Hit":
                # reward = -1000/(target_sell.iloc[step]['delay']+1)
                reward = calculate_optimized_scalping_reward(delay=target_sell.iloc[step]['delay'],action_type="missed_opportunity",consecutive_successes=consecutive_success)
            else:
                reward = 100
            if step+1 < len(df_normalized):
               
                next_state = get_state(df_normalized,step+1)
                
                reward = float(reward)  # Convert to scalar float
                agent.store_transition(state, action, reward, next_state, done)
                agent.update_policy()
            else:
                done = True
                next_state = get_state(df_normalized,-1)
                reward = float(reward)  # Convert to scalar float
                agent.store_transition(state, action, reward, next_state, done)
                agent.update_policy()
            next_step = step+1
            
        pbar.update(next_step - step)
        step = next_step
    pbar.close()
                    
    
    # Update the target network
    if episode % 20 == 0:
        target_net.load_state_dict(policy_net.state_dict())
        model_save_path = os.path.join(save_folder, f'suzlon_14_june_{episode+1}.pth')
        print(f'Episode {episode + 1}')
        print(f'Number of transcations: {number_trans}, Wins: {wins}, Lose: {lose}, Defeat: {defeat} ')

        # Save the model state_dict
        torch.save(policy_net.state_dict(), model_save_path)
       


        

        


100%|██████████| 1473/1473 [00:00<00:00, 88937.33it/s]


Episode 7001
Number of transcations: 35, Wins: 22, Lose: 4, Defeat: 9 


100%|██████████| 1473/1473 [00:00<00:00, 77768.11it/s]


Episode 7021
Number of transcations: 32, Wins: 17, Lose: 4, Defeat: 11 


100%|██████████| 1473/1473 [00:00<00:00, 105536.46it/s]


...

100%|██████████| 1473/1473 [00:08<00:00, 172.17it/s]


Episode 7661
Number of transcations: 37, Wins: 29, Lose: 1, Defeat: 7 


 48%|████▊     | 702/1473 [00:03<00:03, 211.31it/s]

KeyboardInterrupt: 

 49%|████▉     | 728/1473 [00:16<00:03, 211.31it/s]